In [9]:
import numpy as np
import pandas as pd
import datetime
import os

PRICING USING NODE DIVIDEND ADJUSTMENTS

In [10]:
def american_option_price(S0: float, r: float, sigma: float, T: int, K: float, steps: int, opt_type: str, div_sched: dict = {}) -> float:
    """
    Calculates the price of an American Put option using the binomial tree model.
 
    Parameters:
    - S0: float
        The initial stock price.
    - r: float
        The risk-free interest rate.
    - sigma: float
        The volatility of the stock.
    - T: int
        The time to expiration in months.
    - K: float
        The strike price of the option.
    - steps: int
        The number of steps in the binomial tree.
    - opt_type: str
        "call" or "put" type of option
    - div_sched: dict, optional
        A dictionary representing the schedule of discrete dividends.
        Format: {time: amount} where time is the time (in annual units) at which the dividend is paid, and amount is the dividend amount.
 
    Returns:
    - float:
        The price of the American Put option.
 
    Raises:
    - ValueError:
        Will raise an error if any of the input parameters are negative or zero.
    """

    # Validating the input parameters
    if S0 <= 0 or r <= 0 or sigma <= 0 or T <= 0 or K <= 0 or steps <= 0:
        raise ValueError("Input parameters should be positive and non-zero.")

    # Calculating the necessary variables
    dt = T / steps  # Time step size
    u = np.exp(sigma * np.sqrt(dt))  # Up factor
    d = 1 / u  # Down factor
    p = (np.exp(r * dt) - d) / (u - d)  # Probability of up movement

    div_times = [i / dt for i in div_sched.keys()]
    div_amt = list(div_sched.values())
    div_flag = False

    # Creating a 2D array to store the stock prices at each node of the tree
    stock_prices = np.zeros((steps + 1, steps + 1))
    stock_prices[0][0] = S0

    for i in np.arange(1, steps+1):
        div_flag = (div_times != []) and (div_times[0] <= i)
        for j in np.arange(i):
            stock_prices[i][j] = stock_prices[i-1][j] * u
        stock_prices[i][i] = stock_prices[i-1][i-1] * d
        if div_flag:
            stock_prices[i] = np.maximum(stock_prices[i] - div_amt[0], 0)
            div_times.pop(0)
            div_amt.pop(0)

    # Creating a 2D array to store the option values at each node of the tree
    option_values = np.zeros_like(stock_prices)

    # Calculating the option values at each node of the tree
    for i in np.arange(steps, -1, -1):
        for j in np.arange(i + 1):
            intr_val = K - stock_prices[i][j] if opt_type == "put" else stock_prices[i][j] - K
            if i == steps:
                # At the last step, the option value is the maximum of 0 and the intrinsic value
                option_values[i][j] = np.maximum(0, intr_val)
            else:
                # For other steps, the option value is the maximum of the intrinsic value and the discounted expected value
                option_values[i][j] = np.maximum(intr_val,
                    (p * option_values[i + 1][j] + (1 - p) * option_values[i + 1][j + 1]) * np.exp(-r * dt)
                )

    # Returning the option value at the root node of the tree
    return option_values[0][0]


PRICING USING INITIAL DIVIDEND ADJUSTMENT

In [12]:
def american_option_price(S0: float, r: float, sigma: float, T: int, K: float, steps: int, opt_type: str, div_sched: dict = {}) -> float:
    """
    Calculates the price of an American Put option using the binomial tree model.
 
    Parameters:
    - S0: float
        The initial stock price.
    - r: float
        The risk-free interest rate.
    - sigma: float
        The volatility of the stock.
    - T: int
        The time to expiration in months.
    - K: float
        The strike price of the option.
    - steps: int
        The number of steps in the binomial tree.
    - opt_type: str
        "call" or "put" type of option
    - div_sched: dict, optional
        A dictionary representing the schedule of discrete dividends.
        Format: {time: amount} where time is the time (in annual units) at which the dividend is paid, and amount is the dividend amount.
 
    Returns:
    - float:
        The price of the American Put option.
 
    Raises:
    - ValueError:
        Will raise an error if any of the input parameters are negative or zero.
    """

    # Validating the input parameters
    if S0 <= 0 or r <= 0 or sigma <= 0 or T <= 0 or K <= 0 or steps <= 0:
        raise ValueError("Input parameters should be positive and non-zero.")

    # Calculating the necessary variables
    dt = T / steps  # Time step size
    u = np.exp(sigma * np.sqrt(dt))  # Up factor
    d = 1 / u  # Down factor
    p = (np.exp(r * dt) - d) / (u - d)  # Probability of up movement

    # Adjust stock prices for dividends
    if div_sched:
        for time, amount in div_sched.items():
            discount_factor = np.exp(-r * time)
            S0 -= amount * discount_factor

    # Creating a 2D array to store the stock prices at each node of the tree
    stock_prices = np.zeros((steps + 1, steps + 1))

    for i in np.arange(steps+1):
        for j in np.arange(i+1):
            stock_prices[i][j] = S0 * (u ** (i-j)) * (d ** j)

    # Creating a 2D array to store the option values at each node of the tree
    option_values = np.zeros_like(stock_prices)

    # Calculating the option values at each node of the tree
    for i in np.arange(steps, -1, -1):
        for j in np.arange(i + 1):
            intr_val = K - stock_prices[i][j] if opt_type == "put" else stock_prices[i][j] - K
            if i == steps:
                # At the last step, the option value is the maximum of 0 and the intrinsic value
                option_values[i][j] = np.maximum(0, intr_val)
            else:
                # For other steps, the option value is the maximum of the intrinsic value and the discounted expected value
                option_values[i][j] = np.maximum(intr_val,
                    (p * option_values[i + 1][j] + (1 - p) * option_values[i + 1][j + 1]) * np.exp(-r * dt)
                )

    # Returning the option value at the root node of the tree
    return option_values[0][0]


In [13]:
# Example usage:    
S0 = 180
r = 0.02
sigma = 0.25
T = 1
K = 182
steps = 300
div_pmt = 1.56
div_freq = 0.25
# div_sched = dict(zip(np.arange(div_freq, T+div_freq, div_freq), np.repeat(div_pmt, T/div_freq)))
div_sched = {0.5 : 1.56}

opt_type = "call"
option_price = american_option_price(S0, r, sigma, T, K, steps, opt_type, div_sched)
print(f"The price of the American {opt_type.capitalize()} option is: {option_price}")

opt_type = "put"
option_price = american_option_price(S0, r, sigma, T, K, steps, opt_type, div_sched)
print(f"The price of the American {opt_type.capitalize()} option is: {option_price}")

The price of the American Call option is: 17.780885501019494
The price of the American Put option is: 18.080017030503605


In [150]:
for file in os.listdir('../option_data/spy_data'):
    if file[-4:] == '.csv':
                
        df = pd.read_csv('../option_data/spy_data/' + file)        
        
        # moving to datetime and making features
        df['quote_datetime'] = pd.to_datetime(df['quote_datetime'])
        df['expiration'] = pd.to_datetime(df['expiration'])
        df['quote_date'] = df['quote_datetime'][0].date()
        df['quote_date'] = pd.to_datetime(df['quote_date'])
        
        # getting only 4:00 quotes
        eod = datetime.datetime.combine(df['quote_datetime'][0].date(), datetime.time(16,0, 0))
        df = df.loc[df['quote_datetime'] == eod]
        
        # getting time to expiration and moneyness
        df['T'] = df['expiration'] - df['quote_date']
        df['T'] = df['T'].dt.days
        df['moneyness'] = df['active_underlying_price'] / df['strike'] 

        
        # filtering for research paper criteria
        df = df.loc[(df['close']!=0) & (df['implied_volatility']!=0) & (df['T']>=20) & (df['T']<=365) & (df['moneyness']>0.7) & (df['moneyness']<1.3)]

        calls = df.loc[df['option_type']=='C'][['T', 'moneyness', 'implied_volatility', 'active_underlying_price', 'strike']]
        puts = df.loc[df['option_type']=='P'][['T', 'moneyness', 'implied_volatility', 'active_underlying_price', 'strike']]

In [151]:
calls

,T,moneyness,implied_volatility,active_underlying_price,strike
5932,24,1.106442,0.2429,418.235,378.0
5936,24,1.100618,0.2380,418.235,380.0
5948,24,1.086325,0.2272,418.235,385.0
5960,24,1.072397,0.2167,418.235,390.0
5972,24,1.058823,0.2066,418.235,395.0
...,...,...,...,...,...
13036,335,0.853541,0.1333,418.235,490.0
13044,335,0.836470,0.1345,418.235,500.0
13048,335,0.828188,0.1256,418.235,505.0
13060,335,0.804298,0.1267,418.235,520.0
